<a href="https://colab.research.google.com/github/zigzager/SF/blob/main/generador_intervalo_v3.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile visualizador_core.py
# --- ARCHIVO 1: visualizador_core.py ---

# --- Apartado 1: Importaciones del Corazón ---
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# --- Apartado 9: El "Corazón" del Dibujo (Función Interna) ---
def crear_grafico_interno(ruta_csv, tipo_grafico, fecha_inicio, fecha_fin, nombre_archivo_salida):
    """
    Esta es la función "tonta". No pregunta NADA.
    Solo recibe los parámetros y dibuja la imagen.
    """

    # --- Lógica del Apartado 3 (Carga de CSV) ---
    try:
        separador = '\t'
        df = pd.read_csv(
            ruta_csv,
            sep=separador,
            parse_dates={'Datetime': ['<DATE>', '<TIME>']}
        )
        df.rename(columns={
            '<OPEN>': 'Open',
            '<CLOSE>': 'Close',
            '<HIGH>': 'High',
            '<LOW>': 'Low'
        }, inplace=True)
        df.set_index('Datetime', inplace=True)
        df = df[['Open', 'High', 'Low', 'Close']]
    except Exception as e:
        print(f"Error interno (Apartado 3): {e}")
        return

    # --- Lógica del Apartado 5 (Filtrado) ---
    datos_intervalo = df.loc[fecha_inicio : fecha_fin].copy()
    if datos_intervalo.empty:
        print(f"Error interno (Apartado 5): No hay datos para {fecha_inicio}-{fecha_fin}")
        return

    # --- Lógica del Apartado 6 (Dibujo) ---
    # --- ¡CAMBIO AQUÍ! ---
    # Aumentamos la resolución a 300 dpi
    fig, ax = plt.subplots(figsize=(6, 4), dpi=300)
    # --- FIN DEL CAMBIO ---

    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')

    numero_de_puntos = len(datos_intervalo)
    if numero_de_puntos > 300:
        marker_size_m1 = 0.3
        line_width = 0.5
    else:
        marker_size_m1 = 2
        line_width = 1

    x_axis = range(len(datos_intervalo))
    y_open = datos_intervalo['Open'].values
    y_low = datos_intervalo['Low'].values
    y_high = datos_intervalo['High'].values
    y_close = datos_intervalo['Close'].values

    for i in x_axis:
        if y_close[i] >= y_open[i]:
            color_barra = 'lime'
        else:
            color_barra = 'red'

        if tipo_grafico == '1':
            marker_size_actual = marker_size_m1
        else: # H1
            marker_size_actual = 0.3

        ax.vlines(i, ymin=y_low[i], ymax=y_high[i], color=color_barra, linewidth=line_width)
        ax.plot(i, y_close[i], 'o', color=color_barra, markersize=marker_size_actual)

    ax.axis('off')
    ax.margins(0, 0)
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

    # --- Lógica del Apartado 7 (Guardado) ---
    plt.savefig(nombre_archivo_salida, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"\n¡Éxito! Gráfico (Interno) guardado como: '{nombre_archivo_salida}'")
# --- FIN DE visualizador_core.py ---



Overwriting visualizador_core.py


In [2]:
# --- ARCHIVO 2: app.py (corriendo en la celda 2) ---

# --- Apartado 1: Importaciones de la App ---
from datetime import datetime
import uuid
# --- ¡IMPORTANTE! Importamos nuestra propia herramienta ---
from visualizador_core import crear_grafico_interno
# --- FIN APARTADO 1 ---


# --- Apartado 2: Definición de la Función (Interfaz de Usuario) ---
def generar_imagen_intervalo(ruta_csv, formato_fecha, tipo_grafico):
    """
    Esta función es la "Interfaz de Usuario".
    Pregunta las fechas (Apartado 4) y luego llama
    al "corazón" (crear_grafico_interno) para hacer el trabajo.
    """

    # --- APARTADO 4 (DENTRO del Apartado 2) ---
    while True:
        try:
            print(f"\nIntroduce las fechas en formato: {formato_fecha}")
            if formato_fecha == '%Y-%m-%d %H:%M':
                ejemplo_inicio = "2024-10-20 14:30"
                ejemplo_fin = "2024-10-20 16:00"
            else: # Asumimos H1
                ejemplo_inicio = "2024-10-20 14"
                ejemplo_fin = "2024-10-20 20"

            fecha_inicio_str = input(f"Introduce fecha/hora de INICIO (ej: {ejemplo_inicio}): ")
            fecha_inicio = datetime.strptime(fecha_inicio_str, formato_fecha)

            fecha_fin_str = input(f"Introduce fecha/hora de FIN (ej: {ejemplo_fin}):   ")
            fecha_fin = datetime.strptime(fecha_fin_str, formato_fecha)

            if fecha_fin <= fecha_inicio:
                print("Error: La fecha de fin debe ser posterior a la fecha de inicio.")
                continue
            break
        except ValueError:
            print(f"Error: Formato de fecha incorrecto. Debe ser: {formato_fecha}")
    # --- FIN APARTADO 4 ---

    # --- APARTADO 7 (Solo la parte del nombre de archivo) ---
    base_nombre = f"{fecha_inicio.strftime('%Y%m%d%H%M')}_{fecha_fin.strftime('%Y%m%d%H%M')}"
    xyz_aleatorio = uuid.uuid4().hex[:6]
    nombre_archivo_final = f"{base_nombre}_{xyz_aleatorio}.png"

    # --- ¡LLAMADA AL CORAZÓN! ---
    # La Interfaz (app.py) le pasa el trabajo al Corazón (visualizador_core.py)
    crear_grafico_interno(ruta_csv, tipo_grafico, fecha_inicio, fecha_fin, nombre_archivo_final)
# --- FIN APARTADO 2 ---


# --- APARTADO 8: Bloque de Ejecución Principal ---
if __name__ == "__main__":

    # Define las rutas de TUS DOS archivos (súbelos a Colab)
    ruta_m1 = "XAUUSD_M1_202301110812_202511142349.csv"
    ruta_h1 = "XAUUSD_H1_200406110700_202511142300.csv"

    # Bucle principal del Menú
    while True:
        print("\n--- MENÚ PRINCIPAL ---")
        print("¿Qué datos quieres visualizar?")
        print("1: Gráfica de 1 Minuto (M1 - Vela O/C)")
        print("2: Gráfica de 1 Hora (H1 - Vela O/C)")
        print("q: Salir del programa")

        opcion = input("Elige una opción (1, 2, q): ")

        if opcion == '1':
            ruta_a_usar = ruta_m1
            formato_a_usar = '%Y-%m-%d %H:%M'
            tipo_grafico_a_usar = '1'

        elif opcion == '2':
            ruta_a_usar = ruta_h1
            formato_a_usar = '%Y-%m-%d %H'
            tipo_grafico_a_usar = '2'

        elif opcion == 'q' or opcion == 'Q':
            break

        else:
            print("Opción no válida. Por favor, elige 1, 2 o q.")
            continue

        generar_imagen_intervalo(ruta_a_usar, formato_a_usar, tipo_grafico_a_usar)

        while True:
            continuar = input("\n¿Quieres generar otra imagen de esta temporalidad? (s/n): ").lower()
            if continuar == 's':
                generar_imagen_intervalo(ruta_a_usar, formato_a_usar, tipo_grafico_a_usar)
            else:
                break

    print("Programa terminado.")
# --- FIN DE app.py ---


--- MENÚ PRINCIPAL ---
¿Qué datos quieres visualizar?
1: Gráfica de 1 Minuto (M1 - Vela O/C)
2: Gráfica de 1 Hora (H1 - Vela O/C)
q: Salir del programa
Elige una opción (1, 2, q): 1

Introduce las fechas en formato: %Y-%m-%d %H:%M
Introduce fecha/hora de INICIO (ej: 2024-10-20 14:30): 2025-11-14 10:00
Introduce fecha/hora de FIN (ej: 2024-10-20 16:00):   2025-11-14 23:00


/content/visualizador_core.py:18: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(



¡Éxito! Gráfico (Interno) guardado como: '202511141000_202511142300_3dbf65.png'

¿Quieres generar otra imagen de esta temporalidad? (s/n): n

--- MENÚ PRINCIPAL ---
¿Qué datos quieres visualizar?
1: Gráfica de 1 Minuto (M1 - Vela O/C)
2: Gráfica de 1 Hora (H1 - Vela O/C)
q: Salir del programa
Elige una opción (1, 2, q): q
Programa terminado.
